# Multi-Label Text Classification for Stack Overflow Tag Prediction

In multilabel classification output might contain multiple true result. Multiclass for 3 becomes [0 0 1] [1 0 0] etc. Multilabel can be [1 0 1] or [0 0 1] or [1 1 1]

## Load Cleaned Dataset 

dataset: https://raw.githubusercontent.com/laxmimerit/All-CSV-ML-Data-Files-Download/master/stackoverflow.csv

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/laxmimerit/All-CSV-ML-Data-Files-Download/master/stackoverflow.csv', index_col = 0)

In [5]:
df

,Text,Tags
2,aspnet site maps has anyone got experience cre...,"['sql', 'asp.net']"
4,adding scripting functionality to net applicat...,"['c#', '.net']"
5,should i use nested classes in this case i am ...,['c++']
6,homegrown consumption of web services i have b...,['.net']
8,automatically update version number i would li...,['c#']
...,...,...
1262668,using lambda in default initializer gcc vs cla...,['c++']
1262834,stl list very bad performance it is supposed t...,['c++']
1262915,how to use a dict to subset a dataframe say i ...,['python']
1263065,is there a way to use itertools in python to c...,['python']


In [6]:
import ast

In [7]:
df.iloc[0]['Tags']

"['sql', 'asp.net']"

In [8]:
ast.literal_eval(df.iloc[0]['Tags'])

['sql', 'asp.net']

In [9]:
df['Tags'] = df['Tags'].apply(lambda x: ast.literal_eval(x))

In [10]:
df.iloc[0]['Tags']

['sql', 'asp.net']

In [11]:
df.head()

,Text,Tags
2,aspnet site maps has anyone got experience cre...,"[sql, asp.net]"
4,adding scripting functionality to net applicat...,"[c#, .net]"
5,should i use nested classes in this case i am ...,[c++]
6,homegrown consumption of web services i have b...,[.net]
8,automatically update version number i would li...,[c#]


In [12]:
multilabel = MultiLabelBinarizer()
y = multilabel.fit_transform(df['Tags'])

In [13]:
classes = multilabel.classes_

In [14]:
classes

array(['.net', 'android', 'asp.net', 'c', 'c#', 'c++', 'css', 'html',
       'ios', 'iphone', 'java', 'javascript', 'jquery', 'mysql',
       'objective-c', 'php', 'python', 'ruby', 'ruby-on-rails', 'sql'],
      dtype=object)

In [15]:
pd.DataFrame(y, columns=classes)

,.net,android,asp.net,c,c#,c++,css,html,ios,iphone,java,javascript,jquery,mysql,objective-c,php,python,ruby,ruby-on-rails,sql
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48971,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48972,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
48973,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
48974,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [16]:
tfidf = TfidfVectorizer(analyzer='word', max_features = 1000, ngram_range = (1,3), stop_words='english')

In [17]:
X = tfidf.fit_transform(df['Text'])

In [18]:
X

<48976x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 1239765 stored elements in Compressed Sparse Row format>

In [19]:
# tfidf.vocabulary_

1000 Column for X , 20 column for y 

In [21]:
X.shape, y.shape

((48976, 1000), (48976, 20))

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [23]:
X_train.shape, X_test.shape

((39180, 1000), (9796, 1000))

## Model Building

In [24]:
from sklearn.multiclass import OneVsRestClassifier

In [39]:
def j_score(y_true, y_pred):
    jaccard = np.minimum(y_true, y_pred).sum(axis = 1)/np.maximum(y_true, y_pred).sum(axis = 1)
    return jaccard.mean()*100



In [33]:
lr = LogisticRegression(solver='lbfgs')

In [34]:
clf = OneVsRestClassifier(lr)
clf.fit(X_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [35]:
y_pred = clf.predict(X_test)

In [40]:
j_score(y_test, y_pred)

49.12668436096366

## SVM 

In [41]:
from sklearn.svm import LinearSVC

In [42]:
svm = LinearSVC(C = 1.5, penalty='l1', dual = False)
clf = OneVsRestClassifier(svm)
clf.fit(X_train, y_train)

OneVsRestClassifier(estimator=LinearSVC(C=1.5, dual=False, penalty='l1'))

In [43]:
y_pred = clf.predict(X_test)

In [44]:
j_score(y_test, y_pred)

53.3787600381108

In [61]:
x = ['I have used spring framework but flask and django was also okey. Which one would you prefer']

In [62]:
result = clf.predict(tfidf.transform(x))
result

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]])

In [63]:
multilabel.inverse_transform(result)

[('java', 'python')]

# To save the model

In [65]:
import pickle

In [67]:
pickle.dump(clf, open('svm_multilabel.pkl', 'wb'))
pickle.dump(clf, open('tfidf-multilabel.pkl', 'wb'))